<hr>

# LIBRARIES

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

<hr>

# DATASET

In [37]:
df = pd.read_csv("final_df_merged.csv")

In [38]:
pd.set_option('display.max_columns', 10)

* df['y1'] = Kellgren-Lawrence (KL) grades [0,1,2,3,4] (V00XRKL)

* df['y2'] = Knee osteoarthritis (OA), A=non-OA and B=OA (V03KL)

In [39]:
y1_labels = df['y1'].unique().tolist()
y1_labels.sort()
print(y1_labels)
print(type(y1_labels[0]))

[0, 1, 2, 3, 4]
<class 'int'>


In [40]:
df['y2'] = df['y2'].map({'A': 0, 'B': 1})

In [41]:
y2_labels = df['y2'].unique().tolist()
y2_labels.sort()
print(y2_labels)
print(type(y2_labels[0]))

[0, 1]
<class 'int'>


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   case    197 non-null    int64  
 1   d1      197 non-null    float64
 2   d2      197 non-null    float64
 3   d3      197 non-null    float64
 4   d4      197 non-null    float64
 5   d5      197 non-null    float64
 6   d6      197 non-null    float64
 7   d7      197 non-null    float64
 8   d8      197 non-null    float64
 9   d9      197 non-null    float64
 10  d10     197 non-null    float64
 11  d11     197 non-null    float64
 12  d12     197 non-null    float64
 13  d13     197 non-null    float64
 14  d14     197 non-null    float64
 15  d15     197 non-null    float64
 16  d16     197 non-null    float64
 17  d17     197 non-null    float64
 18  d18     197 non-null    float64
 19  d19     197 non-null    float64
 20  d20     197 non-null    float64
 21  d21     197 non-null    float64
 22  d2

In [43]:
df

,case,d1,d2,d3,d4,...,d35,d36,d37,y1,y2
0,9002116,12.754902,12.745283,13.036364,13.434783,...,21.228758,21.947368,23.420000,3,1
1,9005075,31.630952,31.597826,29.600000,27.888889,...,38.690647,39.597122,40.304348,0,0
2,9005132,12.602740,11.950617,11.379310,11.054348,...,17.896296,18.014925,16.230088,4,1
3,9026934,14.588235,14.337079,14.852632,15.080808,...,23.700000,22.131783,23.427481,2,0
4,9030718,31.166667,30.410000,28.771429,27.907407,...,19.689655,17.989247,26.477876,3,1
...,...,...,...,...,...,...,...,...,...,...,...
192,9981798,39.552941,38.187500,33.455446,31.018692,...,17.284553,17.008065,16.637097,1,1
193,9988421,23.387597,23.961832,23.954545,24.045113,...,43.323810,40.659574,38.688889,0,0
194,9990072,18.195122,17.720930,17.872093,16.700000,...,35.794393,32.254717,31.752294,1,0
195,9991313,15.813725,17.398496,18.516393,27.410526,...,27.807692,27.480769,28.670886,4,1


<hr>

# DEFINE X & Y

In [44]:
X = df.iloc[:, 1:-2]  # Features

In [45]:
y1 = df['y1']  # Target variable for KL grades
y2 = df['y2']  # Target variable for OA classification

<hr>
<hr>

# PREDICT 'Y1' (KL GRADES)

<hr>

## train / test / split

In [46]:
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=15)

<hr>

## Models

In [47]:
y1_model_DecisionTree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,
    random_state=16
)

In [48]:
y1_model_RandomForest = RandomForestClassifier(
    n_estimators=200,
    random_state=16
)

In [49]:
y1_model_XGBoost = XGBClassifier(
    objective="multi:softmax",  # outputs class labels directly
    num_class=5,                # number of classes
    n_estimators=300,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="mlogloss",     # avoids warning
    tree_method="hist",         # fast on CPU
    random_state=42
)

In [50]:
y1_model_gnb = GaussianNB()

In [51]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

y1_model_clf = LogisticRegression(
    solver="lbfgs",
    max_iter=1000,
    random_state=42
)

<hr>

## Train models

In [52]:
# Train the model

y1_model_DecisionTree.fit(X_train, y1_train)

,criterion,'gini'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,16
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [53]:
y1_model_RandomForest.fit(X_train, y1_train)

,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [54]:
y1_model_XGBoost.fit(X_train, y1_train)

,objective,'multi:softmax'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'


In [55]:
y1_model_gnb.fit(X_train, y1_train)

,priors,None
,var_smoothing,1e-09


In [56]:
y1_model_clf.fit(X_train_scaled, y1_train)

/Users/wilsonchung/Library/CloudStorage/OneDrive/Documents/__Pace University Courses__/Sem.4 Fall25 [Current]/CS-671-ComputerVision-Fall2025/knee-osteoarthritis-classification-using-bone-distance/myvenv/lib/python3.14/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/wilsonchung/Library/CloudStorage/OneDrive/Documents/__Pace University Courses__/Sem.4 Fall25 [Current]/CS-671-ComputerVision-Fall2025/knee-osteoarthritis-classification-using-bone-distance/myvenv/lib/python3.14/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/wilsonchung/Library/CloudStorage/OneDrive/Documents/__Pace University Courses__/Sem.4 Fall25 [Current]/CS-671-ComputerVision-Fall2025/knee-osteoarthritis-classification-using-bone-distance/my

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


<hr>

## Predictions

In [57]:
y1_pred_DecisionTree = y1_model_DecisionTree.predict(X_test)

In [58]:
y1_pred_RandomForest = y1_model_RandomForest.predict(X_test)

In [59]:
y1_pred_XGBoost = y1_model_XGBoost.predict(X_test)

In [60]:
y1_pred_gnb = y1_model_gnb.predict(X_test)

In [61]:
y1_pred_clf = y1_model_clf.predict(X_test_scaled)

/Users/wilsonchung/Library/CloudStorage/OneDrive/Documents/__Pace University Courses__/Sem.4 Fall25 [Current]/CS-671-ComputerVision-Fall2025/knee-osteoarthritis-classification-using-bone-distance/myvenv/lib/python3.14/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/wilsonchung/Library/CloudStorage/OneDrive/Documents/__Pace University Courses__/Sem.4 Fall25 [Current]/CS-671-ComputerVision-Fall2025/knee-osteoarthritis-classification-using-bone-distance/myvenv/lib/python3.14/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/wilsonchung/Library/CloudStorage/OneDrive/Documents/__Pace University Courses__/Sem.4 Fall25 [Current]/CS-671-ComputerVision-Fall2025/knee-osteoarthritis-classification-using-bone-distance/myvenv/lib/python3.14/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


<hr>

## Accuracy Score

In [62]:
print("y1 DecisionTree Accuracy:", accuracy_score(y1_test, y1_pred_DecisionTree))
print("y1 RandomForest Accuracy:", accuracy_score(y1_test, y1_pred_RandomForest))
print("y1 XGBoost Accuracy:", accuracy_score(y1_test, y1_pred_XGBoost))
print("y1 Gaussian NB Accuracy:", accuracy_score(y1_test, y1_pred_gnb))
print("y1 LogReg Accuracy:", accuracy_score(y1_test, y1_pred_clf))

y1 DecisionTree Accuracy: 0.425
y1 RandomForest Accuracy: 0.3
y1 XGBoost Accuracy: 0.175
y1 Gaussian NB Accuracy: 0.25
y1 LogReg Accuracy: 0.25


<hr>

## Models comparison with loop

In [63]:
y1_models = {
    "DecisionTree": DecisionTreeClassifier(
        criterion='gini',
        max_depth=5,
        random_state=16
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        random_state=16
    ),
    "XGBoost": XGBClassifier(
        objective="multi:softmax",  # outputs class labels directly
        num_class=5,                # number of classes
        n_estimators=300,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="mlogloss",     # avoids warning
        tree_method="hist",         # fast on CPU
        random_state=42
    ),
    "GaussianNB": GaussianNB()
}

In [64]:
for name, model in y1_models.items():
    model.fit(X_train, y1_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y1_test, y_pred)
    print(f"{name}: accuracy = {acc:.3f}")

DecisionTree: accuracy = 0.425
RandomForest: accuracy = 0.300
XGBoost: accuracy = 0.175
GaussianNB: accuracy = 0.250


In [65]:
best_model_y1 = y1_models["DecisionTree"]
y1_pred = best_model_y1.predict(X_test)
print("Model: Multi-class Classification")
print(classification_report(y1_test, y1_pred))

cm = confusion_matrix(y1_test, y1_pred)
print(f"Confusion matrix: \n{cm}\n")

y1_scores = cross_val_score(best_model_y1, X, y1, cv=10,scoring="roc_auc_ovr") # ovr: one-vs-rest / ovo: one-vs-one
y1_10fold_accuracy_scores = y1_scores.tolist()
y1_10fold_accuracy_scores = [round(score, 2) for score in y1_10fold_accuracy_scores]

avg_y1_10fold_accuracy_scores = round(sum(y1_10fold_accuracy_scores) / len(y1_10fold_accuracy_scores), 2)

print("Model: Multi-class Classification")
print("Evaluation metric: AUC score")
print(f"10-fold cross validation scores: \n\t{y1_10fold_accuracy_scores}")
print(f"Average score: {avg_y1_10fold_accuracy_scores}")
print(f"Best score: {max(y1_10fold_accuracy_scores)}")

Model: Multi-class Classification
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.25      0.50      0.33         2
           2       0.37      0.78      0.50         9
           3       0.50      0.36      0.42        11
           4       0.71      0.31      0.43        16

    accuracy                           0.42        40
   macro avg       0.37      0.39      0.34        40
weighted avg       0.52      0.42      0.42        40

Confusion matrix: 
[[0 0 2 0 0]
 [0 1 1 0 0]
 [0 0 7 2 0]
 [1 2 2 4 2]
 [1 1 7 2 5]]

Model: Multi-class Classification
Evaluation metric: AUC score
10-fold cross validation scores: 
	[0.54, 0.59, 0.45, 0.5, 0.51, 0.59, 0.52, 0.47, 0.56, 0.56]
Average score: 0.53
Best score: 0.59


<hr>
<hr>

# PREDICT 'Y2' (A/B)

<hr>

## train / test / split

In [66]:
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=16)

In [67]:
y2_models = {
    "DecisionTree": DecisionTreeClassifier(
        criterion='gini',
        max_depth=5,
        random_state=16
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        random_state=16
    ),
    "XGBoost": XGBClassifier(
        objective="multi:softmax",  # outputs class labels directly
        num_class=5,                # number of classes
        n_estimators=300,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="mlogloss",     # avoids warning
        tree_method="hist",         # fast on CPU
        random_state=42
    ),
    "GaussianNB": GaussianNB()
}

In [68]:
for name, model in y2_models.items():
    model.fit(X_train, y2_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y2_test, y_pred)
    print(f"{name}: accuracy = {acc:.3f}")

DecisionTree: accuracy = 0.575
RandomForest: accuracy = 0.675
XGBoost: accuracy = 0.675
GaussianNB: accuracy = 0.550


In [69]:
best_model_y2 = y2_models["XGBoost"]
y2_pred = best_model_y2.predict(X_test)
print("Model: Binary Classification")
print(classification_report(y2_test, y2_pred))

y2_cm = confusion_matrix(y2_test, y2_pred)
print(f"Confusion matrix: \n{y2_cm}\n")

y2_scores = cross_val_score(best_model_y2, X, y2, cv=10, scoring="roc_auc")
y2_10fold_accuracy_scores = y2_scores.tolist()
y2_10fold_accuracy_scores = [round(score, 2) for score in y2_10fold_accuracy_scores]

avg_y2_10fold_accuracy_scores = round(sum(y2_10fold_accuracy_scores) / len(y2_10fold_accuracy_scores), 2)

print("Model: Binary Classification")
print("Evaluation metric: AUC score")
print(f"10-fold cross validation scores: \n\t{y2_10fold_accuracy_scores}")
print(f"Average score: {avg_y2_10fold_accuracy_scores}")
print(f"Best score: {max(y2_10fold_accuracy_scores)}")

Model: Binary Classification
              precision    recall  f1-score   support

           0       0.38      0.27      0.32        11
           1       0.75      0.83      0.79        29

    accuracy                           0.68        40
   macro avg       0.56      0.55      0.55        40
weighted avg       0.65      0.68      0.66        40

Confusion matrix: 
[[ 3  8]
 [ 5 24]]

Model: Binary Classification
Evaluation metric: AUC score
10-fold cross validation scores: 
	[0.6, 0.89, 0.62, 0.85, 0.6, 0.64, 0.68, 0.74, 0.63, 0.33]
Average score: 0.66
Best score: 0.89
